## Problem 1
### Part (a)

In [1]:
import numpy as np

def gram_schmidt(X):
    # X is an n-by -p matrix .
    # Returns U an orthonormal matrix .
    # eps is a threshold value to identify if a vector
    # is nearly a zero vector .
    eps = 1e-12
    n, p = X.shape
    U = np.zeros((n, 0))
    for j in range(p):
        # Get the j-th column of matrix X
        v = X[:,j]
        # Write your own code here : Perform the
        # orthogonalization by subtracting the projections on
        # all columns of U. And then check whether the vector
        # you get is nearly a zero vector.
        if j == 0:
            u = v / np.linalg.norm(v)
            U = u.reshape(n,1)
            print(U.shape)
        else:
            v = v - (U @ U.T @ v)

            if np.linalg.norm(v) > eps:
                u = v / np.linalg.norm(v)
                U = np.concatenate((U,u.reshape(n,1)),axis = 1)

    return U

### Part (b)

In [2]:
def hilbert_matrix(n):
    X = np.array([[1.0 / (i + j - 1) for i in range(1, n + 1)] for j in range(1, n + 1)])
    return X

In [3]:
hilbert_7 = hilbert_matrix(7)

In [4]:
U = gram_schmidt(hilbert_7)
U

(7, 1)


array([[ 8.13304645e-01, -5.43794290e-01,  1.99095308e-01,
        -5.51132525e-02,  1.19578492e-02, -1.96875818e-03,
        -2.80269164e-04],
       [ 4.06652323e-01,  3.03317262e-01, -6.88560548e-01,
         4.75965265e-01, -1.97392654e-01,  5.41160707e-02,
         5.01279161e-03],
       [ 2.71101548e-01,  3.93949644e-01, -2.07134626e-01,
        -4.90111167e-01,  6.10839673e-01, -3.26896752e-01,
         2.83498633e-03],
       [ 2.03326161e-01,  3.81744394e-01,  1.12389156e-01,
        -4.39598473e-01, -2.54179063e-01,  6.41046772e-01,
        -1.81369762e-01],
       [ 1.62660929e-01,  3.51412668e-01,  2.91518455e-01,
        -1.12276608e-01, -4.99206689e-01, -2.02222128e-01,
         6.01183957e-01],
       [ 1.35550774e-01,  3.20235052e-01,  3.89191824e-01,
         2.30886766e-01, -1.50594721e-01, -5.41817115e-01,
        -7.20509318e-01],
       [ 1.16186378e-01,  2.92095792e-01,  4.40744903e-01,
         5.20623748e-01,  5.01273339e-01,  3.80533243e-01,
         2.9413591

In [5]:
np.linalg.norm(U @ U.T - np.identity(U.shape[0]))

0.37311505015686264

### Part (c)

In [6]:
def modified_gram_schmidt(X):
    # Define a threshold value to identify if a vector
    # is nearly a zero vector .
    eps = 1e-12
    n, p = X.shape
    U = np.zeros((n,0))
    for j in range(p):
        # Get the j-th column of matrix X
        v = X[:, j]
        for i in range(j):
            # Compute and subtract the projection of
            # vector v onto the i-th column of U
            v = v - np.dot(U[:, i], v) * U[:, i]
            # v = np.reshape(v,(-1, 1))
        # Check whether the vector we get is nearly
        # a zero vector
        if np.linalg.norm(v) > eps:
            # Normalize vector v and append it to U
            U = np.hstack((U, (v / np.linalg.norm(v)).reshape(v.shape[0],1)))

    return U

In [7]:
U_mod = modified_gram_schmidt(hilbert_7)

In [8]:
np.linalg.norm(U_mod @ U_mod.T - np.identity(U_mod.shape[0]))

2.7375854036485666e-08

## Problem 6

In [19]:
def cv(X, y, lambdas_, subset_count, fn):
    n, p = X.shape
    subset_size = n // subset_count
    X = X.reshape(subset_count, subset_size, p)
    y = y.reshape(subset_count, subset_size, 1)

    cv_errs = np.zeros(subset_count)
    for i in range(subset_count):
        Xtrain = np.concatenate(np.delete(X,i,axis=0))
        Xtest = X[i]
        ytrain = np.concatenate(np.delete(y,i,axis=0))
        ytest = y[i]

        w = fn(Xtrain, ytrain, lambda_)

        y_hat = Xtest @ w
        preds = np.where(y_hat > 0, 1, -1)
        err = (ytest - preds) / ytest.shape[0]
        cv_errs[i] = err

    return np.mean(cv_errs) * 100

In [20]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import sys
face_data_dict = np.load("face_emotion_data.npz")
X = face_data_dict["X"]
y = face_data_dict["y"]
n, p = np.shape(X)
# error rate for regularized least squares
error_RLS = np.zeros((8, 7))
# error rate for truncated SVD
error_SVD = np.zeros((8, 7))
# SVD parameters to test
k_vals = np.arange(7) + 1
param_err_SVD = np.zeros(len(k_vals))
# RLS parameters to test
lambda_vals = np.array([0, 0.5, 1, 2, 4, 8, 16])
param_err_RLS = np.zeros(len(lambda_vals))

### Part (a)

In [21]:
# truncated svd
def truncSVD(X, y, k):
    y = y.reshape(-1,1)
    n, p = X.shape
    U, Sigma, Vt = np.linalg.svd(X)
    pinv_Sigmak = np.pad(np.diag(np.divide(1, Sigma[:k])),((0,n-k),(0,p-k))).T

    w = Vt.T @ pinv_Sigmak @ U.T @ y

    return w

In [22]:
X = face_data_dict["X"]
y = face_data_dict["y"]
n, p = np.shape(X)
subset_count = 8
subset_size = n // subset_count
X = X.reshape(subset_count, subset_size, p)
y = y.reshape(subset_count, subset_size, 1)

for i in range(subset_count):
    temp = []
    for j in range(subset_count):
        if i != j:
            Xtrain = np.concatenate(np.delete(X,[i,j],axis=0))
            Xval = X[i]
            Xtest = X[j]
            ytrain = np.concatenate(np.delete(y,[i,j],axis=0))
            yval = y[i]
            ytest = y[j]

            param_err_SVD = np.zeros(len(k_vals))
            for num, k in enumerate(k_vals):
                w = truncSVD(Xtrain, ytrain, k)

                y_hat = Xval @ w
                preds = np.where(y_hat > 0, 1, -1)
                err = np.sum(yval - preds) / yval.shape[0]
                param_err_SVD[num] = err

            best_k = k_vals[np.argmin(param_err_SVD)]

            w_best = truncSVD(Xtrain, ytrain, best_k)
            y_hat = Xtest @ w
            preds = np.where(y_hat > 0, 1, -1)
            err = np.sum(np.where(ytest != preds, 1, 0)) / ytest.shape[0]
            temp.append(err)
    error_SVD[i] = np.array(temp)

print("Overall SVD error:", np.round(np.mean(error_SVD) * 100, 2),"%")

Overall SVD error: 4.46 %


### Part (b)

In [23]:
# regularized least squares
def regularizedLS(X, y, lambda_):
    y = y.reshape(-1,1)
    n, p = X.shape
    U, Sigma, Vt = np.linalg.svd(X)
    Sigma_mat = np.pad(np.diag(Sigma),((0,n - len(Sigma)),(0,p - len(Sigma))))

    penal = Sigma_mat.T @ Sigma_mat + lambda_ * np.identity(p)
    w = Vt.T @ np.linalg.inv(penal) @ Sigma_mat.T @ U.T @ y

    return w

In [24]:
X = face_data_dict["X"]
y = face_data_dict["y"]
n, p = np.shape(X)
subset_count = 8
subset_size = n // subset_count
X = X.reshape(subset_count, subset_size, p)
y = y.reshape(subset_count, subset_size, 1)

for i in range(subset_count):
    temp = []
    for j in range(subset_count):
        if i != j:
            Xtrain = np.concatenate(np.delete(X,[i,j],axis=0))
            Xval = X[i]
            Xtest = X[j]
            ytrain = np.concatenate(np.delete(y,[i,j],axis=0))
            yval = y[i]
            ytest = y[j]

            param_err_RLS = np.zeros(len(lambda_vals))
            for num, lambda_ in enumerate(lambda_vals):
                w = regularizedLS(Xtrain, ytrain, lambda_)

                y_hat = Xval @ w
                preds = np.where(y_hat > 0, 1, -1)
                err = np.sum(np.where(ytest != preds, 1, 0)) / ytest.shape[0]
                param_err_RLS[num] = err

            best_lambda = lambda_vals[np.argmin(param_err_RLS)]

            w_best = regularizedLS(Xtrain, ytrain, best_lambda)
            y_hat = Xtest @ w
            preds = np.where(y_hat > 0, 1, -1)
            err = np.sum(np.where(ytest != preds, 1, 0)) / ytest.shape[0]
            temp.append(err)
    error_RLS[i] = np.array(temp)

print("Overall RLS error:", np.round(np.mean(error_RLS) * 100, 2),"%")

Overall RLS error: 5.36 %


In [25]:
error_RLS

array([[0.0625, 0.0625, 0.0625, 0.    , 0.0625, 0.    , 0.    ],
       [0.1875, 0.125 , 0.0625, 0.    , 0.0625, 0.    , 0.    ],
       [0.1875, 0.    , 0.0625, 0.    , 0.0625, 0.    , 0.0625],
       [0.1875, 0.    , 0.0625, 0.    , 0.0625, 0.    , 0.0625],
       [0.1875, 0.    , 0.125 , 0.0625, 0.0625, 0.    , 0.    ],
       [0.1875, 0.    , 0.0625, 0.0625, 0.    , 0.    , 0.    ],
       [0.1875, 0.    , 0.125 , 0.0625, 0.    , 0.0625, 0.    ],
       [0.1875, 0.    , 0.0625, 0.0625, 0.    , 0.0625, 0.    ]])

### Part (c)

In [26]:
X = face_data_dict["X"]
# adding 3 new features
X = np.concatenate((X, X @ np.random.rand(9,3)), axis = 1)
y = face_data_dict["y"]
n, p = np.shape(X)
subset_count = 8
subset_size = n // subset_count
X = X.reshape(subset_count, subset_size, p)
y = y.reshape(subset_count, subset_size, 1)

for i in range(subset_count):
    temp = []
    for j in range(subset_count):
        if i != j:
            Xtrain = np.concatenate(np.delete(X,[i,j],axis=0))
            Xval = X[i]
            Xtest = X[j]
            ytrain = np.concatenate(np.delete(y,[i,j],axis=0))
            yval = y[i]
            ytest = y[j]

            param_err_SVD = np.zeros(len(k_vals))
            for num, k in enumerate(k_vals):
                w = truncSVD(Xtrain, ytrain, k)

                y_hat = Xval @ w
                preds = np.where(y_hat > 0, 1, -1)
                err = np.sum(yval - preds) / yval.shape[0]
                param_err_SVD[num] = err

            best_k = k_vals[np.argmin(param_err_SVD)]

            w_best = truncSVD(Xtrain, ytrain, best_k)
            y_hat = Xtest @ w
            preds = np.where(y_hat > 0, 1, -1)
            err = np.sum(np.where(ytest != preds, 1, 0)) / ytest.shape[0]
            temp.append(err)
    error_SVD[i] = np.array(temp)

print("Overall SVD error:", np.round(np.mean(error_SVD) * 100, 2),"%")

Overall SVD error: 4.58 %


In [27]:
X = face_data_dict["X"]
y = face_data_dict["y"]
n, p = np.shape(X)
subset_count = 8
subset_size = n // subset_count
X = X.reshape(subset_count, subset_size, p)
y = y.reshape(subset_count, subset_size, 1)

for i in range(subset_count):
    temp = []
    for j in range(subset_count):
        if i != j:
            Xtrain = np.concatenate(np.delete(X,[i,j],axis=0))
            Xval = X[i]
            Xtest = X[j]
            ytrain = np.concatenate(np.delete(y,[i,j],axis=0))
            yval = y[i]
            ytest = y[j]

            param_err_RLS = np.zeros(len(lambda_vals))
            for num, lambda_ in enumerate(lambda_vals):
                w = regularizedLS(Xtrain, ytrain, lambda_)

                y_hat = Xval @ w
                preds = np.where(y_hat > 0, 1, -1)
                err = np.sum(np.where(ytest != preds, 1, 0)) / ytest.shape[0]
                param_err_RLS[num] = err

            best_lambda = lambda_vals[np.argmin(param_err_RLS)]

            w_best = regularizedLS(Xtrain, ytrain, best_lambda)
            y_hat = Xtest @ w
            preds = np.where(y_hat > 0, 1, -1)
            err = np.sum(np.where(ytest != preds, 1, 0)) / ytest.shape[0]
            temp.append(err)
    error_RLS[i] = np.array(temp)

print("Overall RLS error:", np.round(np.mean(error_RLS) * 100, 2),"%")

Overall RLS error: 5.36 %
